In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

y_train = pd.read_csv("/content/drive/MyDrive/UCI HAR Dataset/UCI HAR Dataset/train/y_train.txt", delim_whitespace=True, header=None).values.ravel()
y_test = pd.read_csv("/content/drive/MyDrive/UCI HAR Dataset/UCI HAR Dataset/test/y_test.txt", delim_whitespace=True, header=None).values.ravel()


# Combine training and testing data for PCA
final_df1 = pd.read_csv(r"/content/drive/MyDrive/geneticalgooutput.csv")
final_df1 = final_df1.iloc[:, :89]
y_combined_df = pd.concat([pd.Series(y_train), pd.Series(y_test)])


def calculate_similarity(selected_features, ants):
    num_features = len(selected_features)
    if num_features == 0:
        return np.zeros(len(ants[0]))
    similarity = np.sum(ants[:, selected_features], axis=0) / num_features
    similarity = np.pad(similarity, (0, len(ants[0]) - num_features), 'constant')
    return similarity

def aco_feature_selection(num_features, num_ants, num_iterations_aco, X, y):
    pheromones = np.ones(num_features)
    selected_features = []
    temporary=[i for i in range(10299)]
    ants = X[temporary,:]
    ant_fitness1 = np.array([np.mean(ant) for ant in ants])
    for aco_iteration in range(num_iterations_aco):
        ant_fitness=[]
        random_indices=np.random.choice([i for i in range(10299)],size=num_ants)
        for i in random_indices:
            ant_fitness.append(ant_fitness1[i])

        best_ant_index = np.argmax(ant_fitness)

        if 0 <= best_ant_index < 10299:
            best_ant_position = ants[best_ant_index].copy()

            similarity = calculate_similarity(selected_features, ants)
            for feature_i in range(num_features):
                pheromones[feature_i] += ant_fitness[best_ant_index] * (1 - similarity[feature_i])

            pheromone_probs = np.exp(pheromones) / np.sum(np.exp(pheromones))

            selected_feature = np.random.choice(np.arange(num_features), p=pheromone_probs)
            selected_features.append(selected_feature)  # Increment by 1 to match original feature indices

            print(f"ACO Iteration {aco_iteration + 1}/{num_iterations_aco}: Selected feature {selected_feature}")
        else:
            print("Invalid best_ant_index:", best_ant_index)

    return selected_features



X = final_df1
y = y_combined_df

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_final = X_scaled[:]

selected_features_aco = aco_feature_selection(X_final.shape[1], num_ants=200, num_iterations_aco=47, X=X_final, y=y)
selected_features_aco

In [2]:
X_final = X_scaled[:,selected_features_aco]

In [3]:
X_final=pd.DataFrame(X_final)
X_final.head(10299)

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
0,-0.344950,-0.868773,-0.344950,-0.771213,-0.900840,-0.849321,-0.374992,-0.868773,-2.042964,1.485495,...,-0.706356,-1.731883,-0.936732,0.230048,-0.053390,0.918871,-0.344950,1.126231,-1.206949,-3.891042
1,-0.406656,-0.884263,-0.406656,-0.770620,-0.900840,-0.863017,-0.389608,-0.884263,-1.405613,1.269000,...,-0.726777,-1.731546,-0.902697,-0.615071,0.784602,0.908664,-0.406656,0.571360,-1.122341,-1.772857
2,-0.442169,-0.876139,-0.442169,-0.769600,-0.900840,-0.857393,-0.415337,-0.876139,-0.967963,1.347501,...,-0.773143,-1.731210,-0.902697,-2.456876,2.520361,0.892681,-0.442169,0.184032,-1.122341,-0.788590
3,-0.303938,-0.876139,-0.303938,-0.770205,-0.900840,-0.857533,-0.273539,-0.876139,0.358887,1.077732,...,-0.769766,-1.730874,-0.968103,-2.228321,2.380405,0.892681,-0.303938,0.401738,-2.048469,-0.136505
4,-0.395261,-0.883152,-0.395261,-0.771156,-0.900840,-0.859827,-0.396832,-0.883152,0.935553,1.321739,...,-0.770272,-1.730537,-0.943148,-1.305973,1.443147,0.908067,-0.395261,1.129075,-1.639764,-0.970273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10294,0.376249,1.210539,0.376249,0.384289,0.510479,0.337984,0.440705,1.210539,0.735092,-0.332102,...,0.748778,1.730537,0.517569,-0.127378,0.098902,-0.477836,0.376249,-0.769440,0.984414,0.043661
10295,0.277969,1.210539,0.277969,0.856951,0.490641,0.389202,0.311346,1.210539,0.035068,-0.188236,...,0.714089,1.730874,0.961959,-0.007074,-0.013270,-0.310328,0.277969,-0.557853,0.735797,-0.522792
10296,0.213774,0.880306,0.213774,0.669623,0.343827,0.426386,0.183323,0.880306,-0.373917,0.044241,...,0.708206,1.731210,0.961959,0.080272,-0.087762,-0.344355,0.213774,-0.345541,0.884723,-1.030175
10297,-0.095337,0.880306,-0.095337,0.162166,0.198601,0.355007,-0.085267,0.880306,0.050097,0.054367,...,0.681904,1.731546,0.711316,-0.539951,0.548295,-0.470604,-0.095337,-0.407453,1.386772,-1.325299
